In [1]:
# 📌 Install required libraries (only once)
!pip install transformers colorama gTTS --quiet

# 📦 Imports
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from colorama import Fore, Back, Style, init
from gtts import gTTS
from IPython.display import Audio
import datetime
import os
import random
import re

# 🎨 Initialize colorama
init(autoreset=True)
prompt_history = []

# ✨ Banner
def print_banner():
    print(Fore.MAGENTA + Style.BRIGHT + "\n" + "═" * 70)
    print("✨✨ Welcome to InspireMe – AI Text Generator ✨✨".center(70))
    print(Fore.CYAN + "Unleash your creativity with AI-powered storytelling.".center(70))
    print(Style.RESET_ALL + "═" * 70)

# 🚀 Load GPT-2 Model
print(Fore.YELLOW + "🔄 Loading GPT-2 model...")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
print(Fore.GREEN + "✅ Model loaded successfully!\n")

# 📚 Category Selection
def select_category():
    categories = ["Fantasy", "Science Fiction", "Romance", "Adventure", "Horror"]
    print(Fore.YELLOW + "\n📚 Select a category:")
    for idx, category in enumerate(categories, 1):
        print(Fore.CYAN + f"{idx}. {category}")
    choice = input(Fore.YELLOW + "Your choice (1–5): ")
    return categories[int(choice)-1] if choice in map(str, range(1, 6)) else "Fantasy"

# 💡 Prompt Suggestion
def suggest_prompt(category):
    prompts = {
        "Fantasy": ["A magical world where dragons exist...", "A young wizard embarks on a journey..."],
        "Science Fiction": ["In a future where AI controls the world...", "On a spaceship heading to Mars..."],
        "Romance": ["Two people meet under the stars...", "A secret love story unfolds in the city..."],
        "Adventure": ["A treasure map leads to an ancient island...", "Exploring the uncharted forests..."],
        "Horror": ["A haunted house with a dark secret...", "A mysterious figure lurks in the shadows..."]
    }
    return random.choice(prompts.get(category, ["An unknown world awaits..."]))

# 🧠 Text Generation
def generate_text(prompt, max_length=100, temperature=0.9, top_k=50, top_p=0.95):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids, max_length=max_length, temperature=temperature,
        top_k=top_k, top_p=top_p, do_sample=True,
        no_repeat_ngram_size=2, num_return_sequences=1
    )
    result = tokenizer.decode(output[0], skip_special_tokens=True)
    banned = ["irc", "freenode", "kill", "shit", "joined #", "gateway"]
    if any(b in result.lower() for b in banned):
        return Fore.RED + "⚠️ Inappropriate content detected. Try a different prompt."
    return Fore.WHITE + Style.BRIGHT + result

# 💾 Save Story
def save_to_file(text):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"inspire_output_{timestamp}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)
    print(Fore.GREEN + f"\n✅ Output saved to {filename}")

# 🔊 Text-to-Speech
def speak_text(text):
    tts = gTTS(text)
    tts.save("story.mp3")
    return Audio("story.mp3", autoplay=True)

# 📁 View Saved Stories
def list_saved_stories():
    files = [f for f in os.listdir() if f.startswith("inspire_output_") and f.endswith(".txt")]
    if not files:
        print(Fore.RED + "❌ No saved stories found.")
        return
    print(Fore.YELLOW + "\n📂 Saved Stories:")
    for i, f in enumerate(files, 1):
        print(Fore.CYAN + f"{i}. {f}")
    choice = input(Fore.YELLOW + "Enter number to view: ")
    if choice.isdigit() and 1 <= int(choice) <= len(files):
        with open(files[int(choice)-1], "r", encoding="utf-8") as f:
            print(Fore.LIGHTWHITE_EX + f.read())
    else:
        print(Fore.RED + "❌ Invalid choice.")

# 🔍 Search in Saved Stories
def search_saved_stories():
    keyword = input(Fore.YELLOW + "\n🔍 Enter keyword to search in saved stories: ").lower()
    found = False
    for filename in os.listdir():
        if filename.startswith("inspire_output_") and filename.endswith(".txt"):
            with open(filename, "r", encoding="utf-8") as file:
                if keyword in file.read().lower():
                    print(Fore.GREEN + f"✅ Found in {filename}")
                    found = True
    if not found:
        print(Fore.RED + "❌ No stories matched your search.")

# 📋 Format Selector
def customize_text_format():
    formats = {1: "Short Story", 2: "Poem", 3: "Dialogue", 4: "Essay"}
    print(Fore.YELLOW + "\n📋 Select the output format:")
    for k, v in formats.items():
        print(Fore.CYAN + f"{k}. {v}")
    choice = input(Fore.YELLOW + "Your choice (1–4): ")
    return formats.get(int(choice), "Short Story") if choice.isdigit() else "Short Story"

# 🧼 Content Filter
def filter_content(text):
    words = ["kill", "violence", "hate", "death", "blood"]
    if any(re.search(r'\b' + word + r'\b', text.lower()) for word in words):
        return "⚠️ Inappropriate content detected. Try a different prompt."
    return text

# 📊 Count Words and Sentences
def count_words_sentences(text):
    words = len(text.split())
    sentences = len(re.findall(r'[.!?]', text))
    print(Fore.CYAN + f"\n📊 Word Count: {words}, Sentence Count: {sentences}")

# 🕓 Show Prompt History
def show_prompt_history():
    if not prompt_history:
        print(Fore.RED + "❌ No prompt history.")
        return None
    print(Fore.YELLOW + "\n🕓 Prompt History:")
    for idx, p in enumerate(prompt_history, 1):
        print(Fore.CYAN + f"{idx}. {p}")
    idx = input(Fore.YELLOW + "Pick one to reuse (or press Enter to skip): ")
    return prompt_history[int(idx)-1] if idx.isdigit() and 1 <= int(idx) <= len(prompt_history) else None

# ✍️ Optional Editing
def edit_before_saving(original):
    print(Fore.YELLOW + "\n✍️ Do you want to edit the story before saving? (y/n): ", end="")
    if input().lower() == 'y':
        print(Fore.CYAN + "\n🔧 Edit below. Press Enter twice to finish.")
        lines = []
        while True:
            line = input()
            if line == "":
                break
            lines.append(line)
        return "\n".join(lines) if lines else original
    return original

# 🎨 Theme Selector
def set_theme():
    print(Fore.YELLOW + "\n🎨 Choose a theme:\n1. Dark Mode\n2. Light Mode")
    choice = input(Fore.YELLOW + "Your choice (1–2): ")
    if choice == '1':
        os.system('cls' if os.name == 'nt' else 'clear')
        print(Style.BRIGHT + Fore.LIGHTWHITE_EX + Back.BLACK + "\n🌓 Dark Mode Enabled!\n")
    else:
        os.system('cls' if os.name == 'nt' else 'clear')
        print(Fore.BLACK + Back.WHITE + "\n☀️ Light Mode Enabled!\n")

# 🧩 Main App Logic
def inspire_me():
    print_banner()
    set_theme()
    category = select_category()
    print(Fore.YELLOW + f"\n📚 Selected Category: {category}")
    format_choice = customize_text_format()

    while True:
        use_history = input(Fore.YELLOW + "\n🕓 Use a previous prompt? (y/n): ").strip().lower()
        prompt = show_prompt_history() if use_history == 'y' else input(Fore.YELLOW + "\n💡 Enter your creative prompt: ")

        if not prompt:
            prompt = suggest_prompt(category)
            print(Fore.GREEN + f"Suggested prompt: {prompt}")
        prompt_history.append(prompt)

        try:
            max_len = int(input(Fore.CYAN + "🔢 Max text length (e.g., 100): "))
            temperature = float(input(Fore.CYAN + "🌡️ Temperature (0.7–1.3): "))
            top_k = int(input(Fore.CYAN + "🎯 Top-K (e.g., 50): "))
            top_p = float(input(Fore.CYAN + "📊 Top-P (e.g., 0.95): "))
        except ValueError:
            print(Fore.RED + "❌ Invalid input.")
            continue

        print(Fore.BLUE + "\n🛠️ Generating your story...\n")
        result = generate_text(prompt, max_len, temperature, top_k, top_p)
        result = filter_content(result)

        if format_choice == "Poem":
            result = "Poem Format:\n" + result

        print(Fore.LIGHTWHITE_EX + Style.BRIGHT + "\n📝 Your AI Story:\n" + "-"*60)
        print(result)
        print("-"*60)

        count_words_sentences(result)

        if input(Fore.YELLOW + "\n🔊 Listen to the story? (y/n): ").strip().lower() == 'y':
            display(speak_text(result))

        final_result = edit_before_saving(result)

        if input(Fore.BLUE + "\n💾 Save this story? (y/n): ").strip().lower() == 'y':
            save_to_file(final_result)

        if input(Fore.YELLOW + "\n📁 View a saved story? (y/n): ").strip().lower() == 'y':
            list_saved_stories()

        if input(Fore.YELLOW + "\n🔍 Search saved stories? (y/n): ").strip().lower() == 'y':
            search_saved_stories()

        again = input(Fore.MAGENTA + "\n🔁 Try again (r), new prompt (n), or exit (e)? [r/n/e]: ").strip().lower()
        if again == 'r':
            continue
        elif again == 'n':
            os.system('cls' if os.name == 'nt' else 'clear')
            print_banner()
        else:
            print(Fore.CYAN + "\n👋 Thank you for using InspireMe! Keep creating. 🚀")
            break

# ▶️ Start the App
inspire_me()


🔄 Loading GPT-2 model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Model loaded successfully!


══════════════════════════════════════════════════════════════════════
            ✨✨ Welcome to InspireMe – AI Text Generator ✨✨            
        Unleash your creativity with AI-powered storytelling.         
══════════════════════════════════════════════════════════════════════

🎨 Choose a theme:
1. Dark Mode
2. Light Mode
Your choice (1–2): 1

🌓 Dark Mode Enabled!


📚 Select a category:
1. Fantasy
2. Science Fiction
3. Romance
4. Adventure
5. Horror
Your choice (1–5): 1

📚 Selected Category: Fantasy

📋 Select the output format:
1. Short Story
2. Poem
3. Dialogue
4. Essay
Your choice (1–4): 1

🕓 Use a previous prompt? (y/n): n

💡 Enter your creative prompt: Once upon a time
🔢 Max text length (e.g., 100): 100
🌡️ Temperature (0.7–1.3): 1.3
🎯 Top-K (e.g., 50): 50
📊 Top-P (e.g., 0.95): 0.95

🛠️ Generating your story...



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



📝 Your AI Story:
------------------------------------------------------------
Once upon a time people asked why are they talking about a group that's been very bad for black people?" I'd say black women were not talking too many whites because of racism (as long ago as before the Civil Rights movement), and not at all because they didn't want white power (the Clintons probably didn. But that would be silly now about people getting elected and winning elections but not all-white power is bad, but the black woman doesn't actually get as bad as the women of
------------------------------------------------------------

📊 Word Count: 88, Sentence Count: 2

🔊 Listen to the story? (y/n): y



✍️ Do you want to edit the story before saving? (y/n): n

💾 Save this story? (y/n): n

📁 View a saved story? (y/n): n

🔍 Search saved stories? (y/n): n

🔁 Try again (r), new prompt (n), or exit (e)? [r/n/e]: e

👋 Thank you for using InspireMe! Keep creating. 🚀
